In [1]:
import torch 
import pandas as pd 
import torch_geometric as pyg 
import numpy as np 
import os 
from tkgdti.data.GraphBuilder import GraphBuilder
from sklearn.model_selection import KFold

%load_ext autoreload
%autoreload 2

# seed 
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

/home/teddy/miniconda3/envs/tkgdti/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root = '../../extdata/relations/'
OUT = '../../data/tkge_no_patient/'
relnames = os.listdir(root)
N_dtis = pd.read_csv('../../extdata/relations/targetome_drug_targets_gene.csv').shape[0]
K=10
val_prop = 0.075 
NO_REV = False
EXCLUDE_PATIENT_RELATIONS = True

# NOTE (2/28/25): the ablation study suggests that these relations are either detrimental or not beneficial to model performance.
DO_NOT_INCLUDE_RELATIONS=['gene->mut_missense_variant_deleterious_rev->dbgap_subject',
                          'gene->isin_fwd->pathway',
                          'gene->associates_fwd->disease',
                          'gene->A549_lincs_perturbation_rev->drug']


In [3]:
if NO_REV: 
    relnames = [r for r in relnames if '_rev' not in r]

In [4]:
excluded = 0 
relnames2 = [] 
for rname in relnames: 
    rdf = pd.read_csv(root + '/' + rname)

    if rdf.shape[0] == 0:
        print('no relations in df:', rname)
        excluded += 1
        continue 

    if EXCLUDE_PATIENT_RELATIONS: 
        if rdf.src_type.values[0] == 'dbgap_subject' or rdf.dst_type.values[0] == 'dbgap_subject': 
            print('excluding:', rname)
            excluded += 1
            continue
    
    rel_type = f'{rdf.src_type.values[0]}->{rdf.relation.values[0]}->{rdf.dst_type.values[0]}'
    if rel_type in DO_NOT_INCLUDE_RELATIONS: 
        print('excluding:', rname)
        excluded += 1
        continue

    if (rdf.shape[0] > 0): 
        relnames2.append(rname)
relnames = relnames2 

print('# excluded:', excluded)

print('# of relations types:', len(relnames))

no relations in df: beataml_mutation_stop_gained_rel_rev.csv
excluding: beataml_mutation_missense_variant_deleterious_rel_fwd.csv
no relations in df: beataml_mutation_frameshift_variant_rel_fwd.csv
excluding: ctd_genes_diseases_fwd.csv
excluding: A549_lincs_perturbation_rev.csv
excluding: beataml_mutation_missense_variant_tolerated_rel_rev.csv
excluding: beataml_res_rel_rev.csv
excluding: beataml_res_rel_fwd.csv
no relations in df: beataml_mutation_frameshift_variant_rel_rev.csv
excluding: beataml_sens_rel_fwd.csv
excluding: beataml_sens_rel_rev.csv
excluding: ctd_gene_isin_pathway_fwd.csv
excluding: beataml_mutation_missense_variant_deleterious_rel_rev.csv
excluding: beataml_mutation_missense_variant_tolerated_rel_fwd.csv
no relations in df: beataml_mutation_stop_gained_rel_fwd.csv
# excluded: 15
# of relations types: 32


In [5]:

for fold, (train_idxs, test_idxs) in enumerate(KFold(n_splits=K, random_state=seed, shuffle=True).split(range(N_dtis))):
    
    val_idxs = np.random.choice(train_idxs, int(val_prop*N_dtis), replace=False)
    train_idxs = np.array([i for i in train_idxs if i not in val_idxs]) # unnecessary but just to be sure

    assert len(set(train_idxs).intersection(set(val_idxs))) == 0, 'train and val overlap'
    assert len(set(train_idxs).intersection(set(test_idxs))) == 0, 'train and test overlap'
    assert len(set(val_idxs).intersection(set(test_idxs))) == 0, 'val and test overlap'

    print('init...')
    GB = GraphBuilder(root=root, relnames=relnames, val_idxs=val_idxs, test_idxs=test_idxs)
    print('building...')
    GB.build() 
    print('generating triples...')
    train, valid, test, data = GB.get_triples() 

    os.makedirs(f'{OUT}/processed/', exist_ok=True)
    os.makedirs(f'{OUT}/processed/FOLD_{fold}/', exist_ok=True)
    torch.save(train, f'{OUT}/processed/FOLD_{fold}/pos_train.pt')
    torch.save(valid, f'{OUT}/processed/FOLD_{fold}/pos_valid.pt')
    torch.save(test, f'{OUT}/processed/FOLD_{fold}/pos_test.pt')

    # this is deprecated, but may cause issues if not saved. TODO: remove and ensure that the code does not rely on these files
    torch.save(None, f'{OUT}/processed/FOLD_{fold}/neg_train.pt')
    torch.save(None, f'{OUT}/processed/FOLD_{fold}/neg_valid.pt')
    torch.save(None, f'{OUT}/processed/FOLD_{fold}/neg_test.pt')

    torch.save(data, f'{OUT}/processed/FOLD_{fold}/Data.pt')

    print(f'Fold {fold} -> # train: {len(train_idxs)}, # val: {len(val_idxs)}, # test: {len(test_idxs)}')
    print()

init...
Node types: ['disease' 'drug' 'gene' 'pathway']
building...
generating triples...
Fold 0 -> # train: 2666, # val: 242, # test: 324

init...
Node types: ['disease' 'drug' 'gene' 'pathway']
building...
generating triples...
Fold 1 -> # train: 2666, # val: 242, # test: 324

init...
Node types: ['disease' 'drug' 'gene' 'pathway']
building...
generating triples...
Fold 2 -> # train: 2667, # val: 242, # test: 323

init...
Node types: ['disease' 'drug' 'gene' 'pathway']
building...
generating triples...
Fold 3 -> # train: 2667, # val: 242, # test: 323

init...
Node types: ['disease' 'drug' 'gene' 'pathway']
building...
generating triples...
Fold 4 -> # train: 2667, # val: 242, # test: 323

init...
Node types: ['disease' 'drug' 'gene' 'pathway']
building...
generating triples...
Fold 5 -> # train: 2667, # val: 242, # test: 323

init...
Node types: ['disease' 'drug' 'gene' 'pathway']
building...
generating triples...
Fold 6 -> # train: 2667, # val: 242, # test: 323

init...
Node types: 

In [11]:
GB.relations.head()

,dst,src,dst_type,src_type,relation,src_idx,dst_idx
0,QFJCIRLUMZQUOT-HPLJOQBZSA-N,CDK6,drug,gene,PHH_lincs_perturbation_rev,3616,59
1,JOGKUKXHTYWRGZ-UHFFFAOYSA-N,CDK7,drug,gene,PHH_lincs_perturbation_rev,3617,28
2,KTEIFNKAUNYNJU-GFCCVEGCSA-N,IRX5,drug,gene,PHH_lincs_perturbation_rev,10754,35
3,BCFGMOOMADDAQU-UHFFFAOYSA-N,IKZF1,drug,gene,PHH_lincs_perturbation_rev,10381,4
4,XQVVPGYIWAGRNI-JOCHJYFZSA-N,IKZF1,drug,gene,PHH_lincs_perturbation_rev,10381,76


In [12]:
GB.relations.relation.unique()

array(['PHH_lincs_perturbation_rev', 'HT29_lincs_perturbation_rev',
       'PC3_lincs_perturbation_fwd', 'A375_lincs_perturbation_fwd',
       'MCF7_lincs_perturbation_rev', 'interacts_rev',
       'HA1E_lincs_perturbation_rev', 'inhibits_fwd',
       'PC3_lincs_perturbation_rev', 'interacts_fwd', 'isin_rev',
       'HT29_lincs_perturbation_fwd', 'associates_rev',
       'A549_lincs_perturbation_fwd', 'associates_fwd',
       'protbert_similarity', 'inhibits_rev',
       'predicted_strong_binding_rev', 'VCAP_lincs_perturbation_fwd',
       'chemberta_cosine_similarity', 'targets', 'stimulates_fwd',
       'A375_lincs_perturbation_rev', 'SKB_lincs_perturbation_rev',
       'HA1E_lincs_perturbation_fwd', 'SKB_lincs_perturbation_fwd',
       'predicted_strong_binding_fwd', 'VCAP_lincs_perturbation_rev',
       'stimulates_rev', 'MCF7_lincs_perturbation_fwd',
       'PHH_lincs_perturbation_fwd'], dtype=object)

In [13]:
data['num_nodes_dict']

{'disease': 4095, 'drug': 89, 'gene': 26464, 'pathway': 2363}

In [14]:
for k,v in data['node_name_dict'].items(): 
    print(k, len(v))

disease 4095
drug 89
gene 26464
pathway 2363


In [15]:
# report graph metrics

print('num nodes:', sum([v for k,v in data['num_nodes_dict'].items()]))
print('num edges:', sum([v.size(1) for k,v in data['edge_index_dict'].items()]))
print('num node types:', len(data['node_name_dict']))
print('num relations:', len(data['edge_index_dict']))


num nodes: 33011
num edges: 1174279
num node types: 4
num relations: 32
